I (We) certify that the code and data in this assignment were generated independently, 
using only the tools and resources defined in the course and that I (we) did not receive 
any external help, coaching or contributions during the production of this work.

- Arpan Shaileshbhai Korat
- Om Hemant Purohit

In [1]:
from keras.datasets import fashion_mnist
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [2]:
import pandas as pd
tempdf=pd.concat([pd.DataFrame(trainy),pd.DataFrame(testy)])
tempdf["labels"]=tempdf.values.flatten()

# We can see that we have 7k labels for each number from 0 to 9 included

In [ ]:
tempdf.groupby(["labels"]).count()

In [ ]:
tempdf.groupby(["labels"]).count().reset_index()[0]

In [ ]:
import matplotlib.pyplot as plt
gdf=tempdf.groupby(["labels"]).count().reset_index()
plt.bar(gdf["labels"],gdf[0])

In [ ]:
plt.imshow(trainX[0])

0 	T-shirt/top
1 	Trouser
2 	Pullover
3 	Dress
4 	Coat
5 	Sandal
6 	Shirt
7 	Sneaker
8 	Bag
9 	Ankle boot

In [ ]:
def plotClothes(x):
    tempList=[]
    for i in range(len(trainy)):
        if len(tempList)>=4:
            break
        if trainy[i]==x:
            tempList.append(trainX[i])
    fig,ax=plt.subplots(2,2,figsize=(8,8))
    for i in range(4):
        ax[i//2][i%2].imshow(tempList[i])
    




# Plot some t shirts

In [ ]:
plotClothes(0)


# Plot some shoes


In [ ]:
plotClothes(9)

# Plot some sandals

In [ ]:
plotClothes(5)

In [4]:
from keras.datasets import fashion_mnist
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [7]:
trainY.shape

(60000, 10)

In [5]:
trainX=np.expand_dims(trainX,-1)
testX=np.expand_dims(testX,-1)
trainY=keras.utils.to_categorical(trainy,10)
testY=keras.utils.to_categorical(testy,10)


In [ ]:
trainY

In [ ]:
cnn = keras.Sequential([
    keras.Input(shape=(28,28, 1)),
    keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',),
    keras.layers.AveragePooling2D((4, 4)),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu'),

    # keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    # keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(16, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
cnn.compile(optimizer=Adam(lr = 0.00010),
  loss='categorical_crossentropy',
              metrics=['accuracy'])

temp=cnn.fit(trainX,trainY,validation_split=0.2,epochs = 90)

In [ ]:
temp.history.keys()

In [ ]:
cnn.summary()

In [ ]:
import matplotlib.pyplot as plt
basetraining=temp.history["accuracy"]
testing=temp.history["val_accuracy"]
fig,ax=plt.subplots(1,1,figsize=(12,9))
ax.plot(testing,label="testing accuracy")
ax.plot(basetraining,label="training accuracy")
ax.legend()
ax.set_title("Training and testing accuracy plot")
ax.set_xlabel("epochs")

In [ ]:
cnn.evaluate(testX,testY)

In [ ]:
basetrainingLoss=temp.history["loss"]
testing=temp.history["val_loss"]
fig,ax=plt.subplots(1,1,figsize=(12,9))
ax.plot(testing,label="testing loss")
ax.plot(basetraining,label="training loss")
ax.legend()
ax.set_title("Training and testing loss plot")
ax.set_xlabel("epochs")

# EarlyStopping

In [ ]:
from gc import callbacks
cnn.compile(optimizer=Adam(lr = 0.00010),
  loss='categorical_crossentropy',
              metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
cnn.fit(trainX,trainY,validation_split=0.2,epochs = 90,callbacks=[callback])

In [ ]:
cnn.evaluate(testX,testY)

# L1-L2 regularization

In [ ]:
from tensorflow.keras import regularizers
cnn = keras.Sequential([
    keras.Input(shape=(28,28, 1)),
    keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.4, l2=0.2)),
    keras.layers.AveragePooling2D((4, 4)),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.4, l2=0.2)),
    keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.4, l2=0.2)),

    # keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    # keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(16, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax')

])

In [ ]:
cnn.compile(optimizer=Adam(lr = 0.00010),
  loss='categorical_crossentropy',
              metrics=['accuracy'])
cnn.fit(trainX,trainY,validation_split=0.2,epochs = 90)

# K-FOLD Split will not work here as it works for binary values and we are doing multiclass so it will not be applicable here


# Altering Dropout

In [ ]:
dropList=[0.5,0.7,0.9]
testList=[]
for d in dropList:
    
    cnn = keras.Sequential([
        keras.Input(shape=(28,28, 1)),
        keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',),
        keras.layers.AveragePooling2D((4, 4)),

        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu'),

        # keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dropout(d),
        # keras.layers.Dense(32, activation='relu'),
        # keras.layers.Dense(16, activation='sigmoid'),
        keras.layers.Dense(10, activation='softmax')
    ])
    cnn.compile(optimizer=Adam(lr = 0.00010),
      loss='categorical_crossentropy',
                  metrics=['accuracy'])
    cnn.fit(trainX,trainY,validation_split=0.2,epochs = 40)
    testList.append(cnn.evaluate(testX,testY))

# Changing Optimizer

In [ ]:
from tensorflow.keras.optimizers import SGD,Nadam
optList=[Adam(lr = 0.00010),Nadam(lr = 0.00010),SGD(lr=0.00010)]
testList=[]
for d in optList:
    
    cnn = keras.Sequential([
        keras.Input(shape=(28,28, 1)),
        keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',),
        keras.layers.AveragePooling2D((4, 4)),

        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu'),

        # keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        # keras.layers.Dense(32, activation='relu'),
        # keras.layers.Dense(16, activation='sigmoid'),
        keras.layers.Dense(10, activation='softmax')
    ])
    cnn.compile(optimizer=d,
      loss='categorical_crossentropy',
                  metrics=['accuracy'])
    cnn.fit(trainX,trainY,validation_split=0.2,epochs = 40)
    testList.append(cnn.evaluate(testX,testY))

In [ ]:
testList

# Changing Activation Functions

In [ ]:
from tensorflow.keras.optimizers import SGD,Nadam
optList=['relu','selu','tanh']
testList=[]
for d in optList:
    
    cnn = keras.Sequential([
        keras.Input(shape=(28,28, 1)),
        keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation=d,),
        keras.layers.AveragePooling2D((4, 4)),

        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation=d),
        keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation=d),

        # keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        # keras.layers.Dense(32, activation='relu'),
        # keras.layers.Dense(16, activation='sigmoid'),
        keras.layers.Dense(10, activation='softmax')
    ])
    cnn.compile(optimizer=Adam(lr = 0.00010),
      loss='categorical_crossentropy',
                  metrics=['accuracy'])
    cnn.fit(trainX,trainY,validation_split=0.2,epochs = 40)
    testList.append(cnn.evaluate(testX,testY))